In [32]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from tqdm import tqdm, trange
from hepunits import units as u, constants as c
u.dalton = 1.660539068e-27 * u.kg

In [30]:
# Setup parameters
target_width = 1 * u.mm
beam_energy = 1 * u.GeV
beam_current = 100e-6 * u.A
beam_diameter = 2 * u.mm
beam_time = 2 * u.h
decay_time = 4 * u.h
time_step = 60 * u.s
target_material = {'31P': 1.0} # Mole-fractions
target_density = 1.8 * u.g / u.cm3

In [54]:
# Get decay rates and daughters of isotopes

import urllib.request
from functools import lru_cache
import re

@lru_cache(maxsize=None)
def lc_pd_dataframe(**kwargs):
    url = "https://nds.iaea.org/relnsd/v1/data?"
    url += "&".join((f"{k}={v}" for k,v in kwargs.items()))
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req))

In [61]:
all_ground_states = lc_pd_dataframe(fields="ground_states", nuclides="all")

@lru_cache(maxsize=None)
def get_decay_rates(nuclide):
    A, symbol = re.fullmatch("(\d+)(\S+)", nuclide).groups()
    A = int(A)
    ground_state = all_ground_states.query(f"n + z == {A} & symbol == '{symbol}'").iloc[0]
    half_life = ground_state[f"half_life_sec"] * u.s
    if np.isnan(half_life):
        half_life = 1e-99 * u.s
    Z = ground_state["z"]
    N = ground_state["n"]
    daughters = {}
    total_ratio = 0
    for i_d in range(1,4):
        decay_mode = ground_state[f"decay_{i_d}"]
        if not isinstance(decay_mode, str):
            break
        decay_ratio = ground_state[f"decay_{i_d}_%"]
        if np.isnan(decay_ratio):
            decay_ratio = 1e-99
            
        if "B-" in decay_mode:
            d_n = N - 1
            d_z = Z + 1
        elif "B+" in decay_mode or "EC" in decay_mode:
            d_n = N + 1
            d_z = Z - 1
        elif "A" in decay_mode:
            d_n = N - 2
            d_z = Z - 2
        elif "N" in decay_mode:
            d_n = N - 1
            d_z = Z
        elif "P" in decay_mode:
            d_n = N
            d_z = Z - 1
        else:
            break
            
        d_symbol = all_ground_states.query(f"n == {d_n} & z == {d_z}").iloc[0]["symbol"]
        decay = {"d_symbol": d_symbol, "d_n": d_n, "d_z": d_z }
        
        dsym = decay["d_symbol"]
        dA = decay["d_n"] + decay["d_z"]
        daughters[f"{dA}{dsym}"] = decay_ratio
        total_ratio += decay_ratio
    
    for k in daughters:
        # Normalise branching ratios
        daughters[k] /= total_ratio
        # Multiply with decay rate
        daughters[k] *= (np.log(2) / half_life)
    
    return daughters

In [63]:
# Get activation cross sections for isotopes

def get_cross_sections(nuclide):
    A, symbol = re.fullmatch("(\d+)(\S+)", nuclide).groups()
    A = int(A)
    ground_state = all_ground_states.query(f"n + z == {A} & symbol == '{symbol}'").iloc[0]
    half_life = ground_state[f"half_life_sec"] * u.s
    if np.isnan(half_life):
        half_life = 1e-99 * u.s
    Z = ground_state["z"]
    N = ground_state["n"]
    
    # Very basic for now
    p_symbol = all_ground_states.query(f"n == {N} & z == {Z-1}").iloc[0]["symbol"]
    pn_symbol = all_ground_states.query(f"n == {N+1} & z == {Z-1}").iloc[0]["symbol"]
    daughters = {
        f"{A-1}{symbol}": 100 * u.mbarn, # Kick out a neutron
        f"{A-1}{p_symbol}": 200 * u.mbarn, # Kick out a proton
        f"{A}{pn_symbol}": 70e-42 * u.cm2, # Turn a proton into a neutron
    }
    

In [50]:
# Initial number of isotopes
target_volume = c.pi*(beam_diameter/2)**2 * target_width
target_mass = target_volume * target_density
target_nucleons = target_mass / u.dalton

start_isotopes = {}
for nuc, rel in target_material.items():
    start_isotopes[nuc] = target_nucleons * rel

start_isotopes

{'31P': 3.4054403689956597e+21}

In [39]:
# Simulate the irradiation
beamsteps = int(beam_time / time_step)
current_isotopes = start_isotopes
for _ in trange(beamsteps):
    pass

100%|██████████████████████████████████████████████████████████████████████████████| 120/120 [00:00<00:00, 1413810.34it/s]
